In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
ANIME_DIR = "anime_data/"

## Предобработка данных

### работа с признаками

In [ ]:
anime_ratings = pd.read_csv(ANIME_DIR + "animelist.csv", nrows=10000000)
anime_data = pd.read_csv(ANIME_DIR + "anime.csv")

Рассиморим, какая информация находится в файле $anime.csv$

In [ ]:
anime_data.info()

Сразу переименуем колонку $MAL\_ID$ в $anime\_id$

In [ ]:
anime_data.rename(columns={'MAL_ID':"anime_id"},inplace=True)

In [ ]:
anime_data.columns

Избавимся от лишних полей

In [ ]:
to_keep = ['anime_id', 'Name', 'Score', 'Genres', 'Members']
anime_data = anime_data[to_keep]
anime_data

Сейчас у нас все жанры описаны в одном поле через запятую, что не очень удобно, поэтому мы определим все жанры и добавим их как поля для каждой записи

In [ ]:
genres_column = anime_data["Genres"].map(lambda x: x.split(", "))
genres = list(set(sum(genres_column, [])))

anime_data[genres] = 0
for i in range(0, len(genres_column)):
    anime_data.loc[i, genres_column[i]] = 1

anime_data = anime_data.drop(columns="Genres")

Заменим Unknown в поле Score на 0

In [ ]:
dict = {'Unknown' : 0}
anime_data['Score'] = anime_data['Score'].astype(str).apply(lambda x : dict[x] if x == 'Unknown' else x).astype(float)

anime_data.info()

Рассмотрим теперь информацию файла $animelist.csv$

In [ ]:
anime_ratings.info()

Избавимся от информации о кол-ве просмотренных эпизодов и статусе просмотра

In [ ]:
anime_ratings = anime_ratings[['user_id', 'anime_id', 'rating']]
anime_ratings.info()

Проверим, есть ли оценки для всех аниме, представленных в датасете

In [ ]:
anime_ratings.anime_id.nunique()

Это действительно так

Объеденим информацию из двух файлов

In [ ]:
anime_complete = pd.merge(anime_data, anime_ratings, on='anime_id')
anime_complete.info()

Переименуеем Score d total_score, а rating в user_score

In [ ]:
anime_complete = anime_complete.rename(columns={'Score' : 'total_score', 'rating': 'user_score'})

anime_complete.isna().sum()

сохраним полученый df в csv

In [ ]:
anime_complete.to_csv(ANIME_DIR + 'complete.csv')

### Подготовка данных для рекомендаций

In [ ]:
anime_feature = pd.read_csv(ANIME_DIR + 'complete.csv')
anime_feature = anime_feature.drop(columns='Unnamed: 0')
anime_feature.head()

Ради интереса посмотрим на 10 самых популярных аниме по кол-ву оценок и по кол-ву фанатов

In [ ]:
top10_by_score = anime_feature['Name'].value_counts().nlargest(10)
top10_by_members = anime_feature.sort_values(by='Members', ascending=False).drop_duplicates(subset='Name').head(10)

In [ ]:
def draw_barplor(df_x, df_y, title, xlabel, ylabel):
    plt.bar(df_x, df_y)

    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xticks(rotation=30, ha="right")

    plt.show()

In [ ]:
draw_barplor(top10_by_score.index, top10_by_score.values, 
             "топ 10 по суммарному рейтингу", "название аниме", "суммарный рейтинг")

И тут в поезде анимешников завязалась драка...

In [ ]:
draw_barplor(top10_by_members['Name'], top10_by_members['Members'],
             "топ 10 по числу фанатов", "название аниме", "фанаты")

...в которую ворвались адепты всемогущего, Сайтамы и 1000 - 7 ...

Проверим сколько в среднем поставил оценок каждый пользователь

In [ ]:
count_of_users = anime_feature['user_id'].value_counts()
count_of_users.describe()

Половина пользователей, оцекни которых мы собираемся использовать поставили
оценки меньше чем $67$ анииме, однако средним значением для выборки является около
$101$-ой оценки. Если рассматривать, что большая часть оценок поставлена по
просмотру аниме и при этом не для каждого просмотренного аниме пользователь
поставил оценку, то на мой взгляд выбрать для дальнейшей работы
стоит выбрать пользователей, которые поставили оцеку $75$ и более аниме. 

Данное значение было взято из следующего:
- Мной просмотрено около 300 аниме, но при этом оценка выставлена лишь половине.
- Большая часть выходящих аниме преставляют собой 12-ти серийные сериалы,
  средняя продолжительность которых составляет $12 * 24 / 60 = 4.8$ часа.
- Следовательно человек, просмотревший $75$ аниме, потратил на это $360$ часов...

$360$ часов можно интерпретировать примерно как год работы кинокритика, т.к
кроме самого просмотра, человек ещё тратит какое-то время на осмысление сюжета,
понимание мотивов героев. Если он конечно смотрит их не залпом по несколько
аниме в день из-за кошкодевочек и им подобным)

In [ ]:
print(f'{type(count_of_users)}\n{count_of_users}')

In [ ]:
anime_feature = anime_feature[anime_feature['user_id'].isin(count_of_users[count_of_users >= 75].index)]
anime_feature.user_id.nunique()

создадим теперь таблицу, в которой строками будут названия аниме, а столбцами id пользователей. Значениями будут оценки

In [ ]:
anime_pivot = anime_feature.pivot_table(index='Name', columns='user_id',
                                        values='user_score').fillna(0)
anime_pivot.head()

также сохраним его для дальнейшей работы

In [ ]:
anime_pivot.to_csv(ANIME_DIR + 'anime_cf.csv')